In [111]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime, date
from dateutil.relativedelta import relativedelta

from sklearn.preprocessing import StandardScaler

from math import ceil

from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger, EarlyStopping
from keras.optimizers import RMSprop, Adam, SGD, Nadam
from keras.layers.advanced_activations import *
from keras.layers import Convolution1D, MaxPooling1D, AtrousConvolution1D
from keras.layers.recurrent import LSTM, GRU
from keras import regularizers

%matplotlib inline


In [112]:
train = pd.read_csv('all/sales_train.csv')
test = pd.read_csv('all/test.csv')
submission = pd.read_csv('all/sample_submission.csv')
items = pd.read_csv('all/items.csv')
item_cats = pd.read_csv('all/item_categories.csv')
shops = pd.read_csv('all/shops.csv')
print("Ok")

Ok


In [113]:
test_shops = test.shop_id.unique()
train = train[train.shop_id.isin(test_shops)]
test_items = test.item_id.unique()
train = train[train.item_id.isin(test_items)]

MAX_BLOCK_NUM = train.date_block_num.max()
MAX_ITEM = len(test_items)
MAX_CAT = len(item_cats)
MAX_YEAR = 3
MAX_MONTH = 4 # 7 8 9 10
MAX_SHOP = len(test_shops)

In [114]:
# add categories
train = train.set_index('item_id').join(items.set_index('item_id')).drop('item_name', axis=1).reset_index()


train['month'] = train.date.apply(lambda x: datetime.strptime(x, '%d.%m.%Y').strftime('%m'))
train['year'] = train.date.apply(lambda x: datetime.strptime(x, '%d.%m.%Y').strftime('%Y'))

In [115]:
train = train.drop('date', axis=1)
train = train.drop('item_category_id', axis=1)
train = train.groupby(['shop_id', 'item_id', 'date_block_num', 'month', 'year']).sum()
train = train.sort_index()


In [116]:
scaler = StandardScaler()
cnt_scaler = StandardScaler()

scaler.fit(train.item_price.values.reshape(-1, 1))
cnt_scaler.fit(train.item_cnt_day.values.reshape(-1, 1))

train.item_price = scaler.transform(train.item_price.values.reshape(-1, 1))
# train.item_cnt_day = cnt_scaler.transform(train.item_cnt_day.values.reshape(-1, 1))

In [117]:
price = train.reset_index().set_index(['item_id', 'shop_id', 'date_block_num'])
price = price.sort_index()

In [118]:
from sklearn import preprocessing

item_le = preprocessing.LabelEncoder()
item_le.fit(test_items)
item_dm = dict(zip(test_items, item_le.transform(test_items)))


shop_le = preprocessing.LabelEncoder()
shop_le.fit(test_shops)
shop_dm = dict(zip(test_shops, shop_le.transform(test_shops)))


month_le = preprocessing.LabelEncoder()
month_le.fit(range(7,11))
month_dm = dict(zip(range(7,11), month_le.transform(range(7,11))))

In [119]:
values = train.values.tolist()
keys = train.index.values.tolist()
table = []
for i in range(len(item_dm)):
    table.append([])
    for j in range(len(shop_dm)):
        table[i].append([])
        for k in range(34):
            table[i][j].append([0,10])
print("prepared")

prepared


In [120]:
values = train.values.tolist()
keys = train.index.values.tolist()
for i in range(len(train.index)):
    key = keys[i]
    value = values[i]
    table[item_dm[key[1]]][shop_dm[key[0]]][key[2]] = [value[0],value[1]]


In [121]:
for i in range(len(item_dm)):
    table.append([])
    for j in range(len(shop_dm)):
        price = 0
        count = 0
        k = 0
        while price==0 and k < 34:
            if table[i][j][k][1]<9: 
                price = table[i][j][k][0]
            k = k+1
        k = 0
        while k < 34:
            if table[i][j][k][1]<9: 
                price = table[i][j][k][0]
            else:
                table[i][j][k][1] = 0
                table[i][j][k][0] = price
            k = k+1
print("done")
# table[0][0]

done


In [144]:
res = []
maxx = 0
for i in range(len(item_dm)):
    res.append([])
    for j in range(len(shop_dm)):
        res[i].append([])
        summ = 0
        for k in range(34):
            summ = summ+table[i][j][k][1]
        res[i][j] = summ
        if summ>maxx:
            maxx=summ


AttributeError: 'list' object has no attribute 'plot'

In [122]:
MAX_ITEM = len(test_items)
MAX_MONTH = 32 
MAX_SHOP = len(test_shops)

In [128]:

# x_train = np.zeros((MAX_ITEM,MAX_SHOP*MAX_MONTH*2), dtype=np.float32)
# for i, item in enumerate(table):
#     for j, sales in enumerate(item):  
#         for k in range(MAX_MONTH):
#             x_train[i][j*MAX_MONTH*2+k*2] = table[i][j][k][0]
#             x_train[i][j*MAX_MONTH*2+k*2+1] = table[i][j][k][1]
            
# y_train = np.zeros((MAX_ITEM,MAX_SHOP), dtype=np.float32)
# for i, item in enumerate(table):
#     for j, sales in enumerate(item):  
#         y_train[i][j] = table[i][j][MAX_MONTH][1]

x_train = np.zeros((MAX_ITEM, MAX_MONTH, MAX_SHOP*2), dtype=np.float32)
for i, item in enumerate(table):
    for j, sales in enumerate(item):  
        for k in range(MAX_MONTH):
            x_train[i][k][j*2] = table[i][j][k][0]
            x_train[i][k][j*2+1] = table[i][j][k][1]
            
# y_train = np.zeros((MAX_ITEM,MAX_SHOP), dtype=np.float32)
# for i, item in enumerate(table): 
#      for j, sales in enumerate(item):  
#         y_train[i][j] = table[i][j][MAX_MONTH][1]
y_train = np.zeros((MAX_ITEM,MAX_SHOP*5), dtype=np.float32)
for i, item in enumerate(table): 
    for j, sales in enumerate(item):
        val = table[i][j][MAX_MONTH][1]
        if val>=5:
            val = 4
        y_train[i][int(j*5+val)] = 1

In [ ]:
# x_train[0][0]

In [129]:
# x_val = np.zeros((MAX_ITEM,MAX_SHOP*MAX_MONTH*2), dtype=np.float32)
# for i, item in enumerate(table):
#     for j, sales in enumerate(item):  
#         for k in range(MAX_MONTH):
#             x_val[i][j*MAX_MONTH*2+k*2] = table[i][j][k+1][0]
#             x_val[i][j*MAX_MONTH*2+k*2+1] = table[i][j][k+1][1]
            


# y_val = np.zeros((MAX_ITEM,MAX_SHOP), dtype=np.float32)
# for i, item in enumerate(table):
#     for j, sales in enumerate(item):  
#         y_val[i][j] = table[i][j][MAX_MONTH+1][1]

x_val = np.zeros((MAX_ITEM, MAX_MONTH, MAX_SHOP*2), dtype=np.float32)
for i, item in enumerate(table):
    for j, sales in enumerate(item):  
        for k in range(MAX_MONTH):
            x_val[i][k][j*2] = table[i][j][k+1][0]
            x_val[i][k][j*2+1] = table[i][j][k+1][1]
            
y_val = np.zeros((MAX_ITEM,MAX_SHOP*5), dtype=np.float32)
for i, item in enumerate(table): 
     for j, sales in enumerate(item):  
        val = table[i][j][MAX_MONTH+1][1]
        if val>=5:
            val = 4
        y_train[i][int(j*5+val)] = 1
#         y_val[i][j] = table[i][j][MAX_MONTH+1][1]

In [130]:
print(len(x_train[0]), len(x_train[0][0]))

32 84


In [132]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
# model.add(LSTM(1, input_shape=(maxlen, length)))
# model.add(Dense(1, activation='relu'))
model.add(LSTM(32, return_sequences=True, input_shape=(32, 84),recurrent_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(LSTM(32, return_sequences=True, input_shape=(32, 84),recurrent_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(LeakyReLU())
# model.add(Dropout(0.3))
# model.add(LSTM(32, return_sequences=True, input_shape=(32, 84)))
model.add(LSTM(32, return_sequences=False, input_shape=(32, 84),recurrent_initializer='he_uniform'))
# model.add(BatchNormalization())
# model.add(LeakyReLU())
# model.add(Dense(1, activation='relu'))

# model.add(Dense(64, input_dim=(2688), kernel_initializer='he_uniform'))
# model.add(BatchNormalization())
# model.add(LeakyReLU())
# model.add(Dropout(0.5))
# model.add(Dense(32))
# model.add(BatchNormalization())
# model.add(LeakyReLU())
# model.add(Dropout(0.5))
# model.add(Dense(64))
# model.add(BatchNormalization())
# model.add(LeakyReLU())
# model.add(Dropout(0.5))

# model.add(Dense(128, input_dim=(2688)))
# model.add(Dense(64))
# model.add(Dense(64))

# model.add(Dense(256))
# model.add(BatchNormalization())
# model.add(LeakyReLU())
# model.add(Dropout(0.3))

# model.add(Dense(16))
# model.add(BatchNormalization())
# model.add(LeakyReLU())
model.add(Dense(MAX_SHOP*5, activation='softmax'))
# model.add(Activation('softmax'))


# from keras.optimizers import RMSprop, Adam, SGD, Nadam
# optimizer = RMSprop(lr=0.005)
# optimizer = SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)
# optimizer = Nadam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
optimizer = Adam(lr=0.005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

model.compile(loss='mean_squared_error', optimizer=optimizer)

# model.fit(x_train, y_train, batch_size=2048, epochs=50, validation_data=(x_val, y_val))
model.fit(x_train, y_train, batch_size=2048, epochs=50, validation_data=(x_val, y_val))
# model.fit(x_train, y_train, batch_size=2048, epochs=10)

Build model...
Train on 5100 samples, validate on 5100 samples
Epoch 1/50
5100/5100 [==============================] - 18s 4ms/step - loss: 0.2332 - val_loss: 2.3534e-05
Epoch 2/50
5100/5100 [==============================] - 1s 145us/step - loss: 0.2331 - val_loss: 2.4208e-05
Epoch 3/50
5100/5100 [==============================] - 1s 149us/step - loss: 0.2327 - val_loss: 2.6841e-05
Epoch 4/50
5100/5100 [==============================] - 1s 148us/step - loss: 0.2319 - val_loss: 3.9298e-05
Epoch 5/50
5100/5100 [==============================] - 1s 148us/step - loss: 0.2303 - val_loss: 7.4824e-05
Epoch 6/50
5100/5100 [==============================] - 1s 151us/step - loss: 0.2284 - val_loss: 1.1193e-04
Epoch 7/50
5100/5100 [==============================] - 1s 150us/step - loss: 0.2272 - val_loss: 1.3698e-04
Epoch 8/50
5100/5100 [==============================] - 1s 145us/step - loss: 0.2268 - val_loss: 1.4768e-04
Epoch 9/50
5100/5100 [==============================] - 1s 145us/step - lo

In [133]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_52 (LSTM)               (None, 32, 32)            14976     
_________________________________________________________________
batch_normalization_111 (Bat (None, 32, 32)            128       
_________________________________________________________________
leaky_re_lu_110 (LeakyReLU)  (None, 32, 32)            0         
_________________________________________________________________
lstm_53 (LSTM)               (None, 32, 32)            8320      
_________________________________________________________________
batch_normalization_112 (Bat (None, 32, 32)            128       
_________________________________________________________________
leaky_re_lu_111 (LeakyReLU)  (None, 32, 32)            0         
_________________________________________________________________
lstm_54 (LSTM)               (None, 32)                8320      
__________

In [134]:
# MAX_ITEM = len(test_items)
# MAX_MONTH = 32 
# MAX_SHOP = len(test_shops)

# month_count = 32
# x_test = np.zeros((MAX_ITEM,MAX_SHOP*MAX_MONTH*2), dtype=np.float32)
# for i, item in enumerate(table):
#     for j, sales in enumerate(item):  
#         for k in range(MAX_MONTH):
#             x_test[i][j*MAX_MONTH*2+k*2] = table[i][j][k+2][0]
#             x_test[i][j*MAX_MONTH*2+k*2+1] = table[i][j][k+2][1]
            
            
x_test = np.zeros((MAX_ITEM, MAX_MONTH, MAX_SHOP*2), dtype=np.float32)
for i, item in enumerate(table):
    for j, sales in enumerate(item):  
        for k in range(MAX_MONTH):
            x_test[i][k][j*2] = table[i][j][k+2][0]
            x_test[i][k][j*2+1] = table[i][j][k+2][1]

In [106]:
y_train[0]

array([ 0.        ,  0.        ,  0.        ,  0.        , -0.15018865,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        , -0.15018865,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ], dtype=float32)

In [135]:
predict_test = model.predict(x_test)

In [137]:
predict_test[0]

array([3.40062827e-02, 1.79483891e-06, 1.32155208e-06, 2.44939429e-06,
       2.20105699e-06, 4.23374288e-02, 1.83745851e-06, 1.08320864e-06,
       1.72728187e-06, 1.84071371e-06, 3.61154303e-02, 3.79306675e-06,
       2.01508033e-06, 1.98901853e-06, 1.76862602e-06, 2.47798003e-02,
       2.36423989e-06, 2.27325768e-06, 2.30488490e-06, 2.06634149e-06,
       7.56919151e-03, 5.65094115e-06, 1.86198167e-06, 1.19420065e-06,
       2.41746898e-06, 1.39330439e-02, 2.53451503e-06, 2.13967837e-06,
       1.65792335e-06, 1.03226910e-06, 5.16091585e-02, 3.67291796e-06,
       2.16585227e-06, 1.95733219e-06, 1.60680406e-06, 2.84844898e-02,
       1.57260604e-06, 1.40646011e-06, 2.07964854e-06, 2.26017505e-06,
       3.56795639e-02, 2.47934258e-06, 1.46514378e-06, 1.15042576e-06,
       2.03657510e-06, 1.65591370e-02, 1.82980614e-06, 1.63527955e-06,
       1.13816100e-06, 2.50957805e-06, 2.48355865e-02, 1.82418182e-06,
       1.52385871e-06, 2.27727855e-06, 1.60379420e-06, 1.91739853e-02,
      

In [100]:
predict_test = cnt_scaler.inverse_transform(predict_test)

In [101]:
predict_test[0][0]

2.6955707

In [104]:
predict_test[0]

array([2.6955707, 2.6955707, 2.6955707, 2.6955707, 2.6955707, 2.6955707,
       2.6955707, 2.6955707, 2.6955707, 2.6955707, 2.6955707, 2.6955707,
       2.6955707, 2.6955707, 2.6955707, 2.6955707, 2.6955707, 2.6955707,
       2.6955707, 2.6955707, 2.6955707, 2.6955707, 2.6955707, 2.6955707,
       2.6955707, 2.6955707, 2.6955707, 2.6955707, 2.6955707, 2.6955707,
       2.6955707, 2.6955707, 2.6955707, 2.6955707, 2.6955707, 2.6955707,
       2.6955707, 2.6955707, 2.6955707, 2.6955707, 2.6955707, 2.6955707],
      dtype=float32)

In [167]:
summ = 0 

# for i in range(len(item_dm)):
# #     res.append([])
#     for j in range(len(shop_dm)):
# #         res[i].append([])
# #         summ = 0
# #         for k in range(34):
# #             summ = summ+table[i][j][k][1]
#         summ = table[i][j][33][1] + summ
# #         if summ>maxx:
# #             maxx=summ

# print(summ)
# summ = 0
for k in range(6):
    for sentence in test2.index.values.tolist():
        (shop_id, item_id) = (sentence[0], sentence[1])
        iid = item_dm[item_id]
        sid = shop_dm[shop_id]
        summ = summ + table[iid][sid][28+k][1]
        
print(summ)

summ = 0
for sentence in test2.index.values.tolist():
        (shop_id, item_id) = (sentence[0], sentence[1])
        iid = item_dm[item_id]
        sid = shop_dm[shop_id]
        summ = summ + table[iid][sid][34-12][1]
print(summ)


252950.0
38947.0


In [177]:
test = pd.read_csv('all/test.csv')
test = test.set_index(['shop_id', 'item_id'])
test['item_cnt_month'] = 0

test2 = test
test2
summ = 0
for index, sentence in enumerate(test2.index.values.tolist()):
#     print(index, sentence)
    (shop_id, item_id) = (sentence[0], sentence[1])
    iid = item_dm[item_id]
    sid = shop_dm[shop_id]
    pr1 = 0
    for k in range(6):
        (shop_id, item_id) = (sentence[0], sentence[1])
        iid = item_dm[item_id]
        sid = shop_dm[shop_id]
        pr1 = pr1 + table[iid][sid][28+k][1]
        
    pr2 = 0
    (shop_id, item_id) = (sentence[0], sentence[1])
    iid = item_dm[item_id]
    sid = shop_dm[shop_id]
    pr2 = pr2 + table[iid][sid][34-11][1]
    pr2 = pr2 + table[iid][sid][34-12][1]
    pr2 = pr2 + table[iid][sid][34-13][1]
       
    
#     val = int(res[iid][sid]/120.0+pr1/35.0+pr2/25.0)
    val = int(res[iid][sid]/40.0+pr1/18.0+pr2/6.0)
    
    
#     test.loc[(shop_id, item_id)]['item_cnt_month'] = predict_test[item_dm[item_id]][shop_dm[shop_id]]
    test.loc[(shop_id, item_id)]['item_cnt_month'] = val
    summ=summ+val

print(summ)
    

test = test.reset_index().drop(['shop_id', 'item_id'], axis=1)
test.to_csv('submission2.csv', index=False)


28072


In [ ]:
predict = []
for i in range(len(item_dm)):
    res.append([])
    for j in range(len(shop_dm)):
        res[i].append([])
        

for index, sentence in enumerate(test2.index.values.tolist()):
#     print(index, sentence)
    (shop_id, item_id) = (sentence[0], sentence[1])
    iid = item_dm[item_id]
    sid = shop_dm[shop_id]
    pr1 = 0
    for k in range(6):
        (shop_id, item_id) = (sentence[0], sentence[1])
        iid = item_dm[item_id]
        sid = shop_dm[shop_id]
        pr1 = pr1 + table[iid][sid][28+k][1]
        
    pr2 = 0
    (shop_id, item_id) = (sentence[0], sentence[1])
    iid = item_dm[item_id]
    sid = shop_dm[shop_id]
    pr2 = pr2 + table[iid][sid][34-11][1]
    pr2 = pr2 + table[iid][sid][34-12][1]
    pr2 = pr2 + table[iid][sid][34-13][1]
       
    
#     val = int(res[iid][sid]/120.0+pr1/35.0+pr2/25.0)
    val = int(res[iid][sid]/40.0+pr1/18.0+pr2/6.0)
    
    
#     test.loc[(shop_id, item_id)]['item_cnt_month'] = predict_test[item_dm[item_id]][shop_dm[shop_id]]
    test.loc[(shop_id, item_id)]['item_cnt_month'] = val
    summ=summ+val

print(summ)
    

test = test.reset_index().drop(['shop_id', 'item_id'], axis=1)
test.to_csv('submission2.csv', index=False)

In [165]:
summ = 0 

# for i in range(len(item_dm)):
# #     res.append([])
#     for j in range(len(shop_dm)):
# #         res[i].append([])
# #         summ = 0
# #         for k in range(34):
# #             summ = summ+table[i][j][k][1]
#         summ = table[i][j][33][1] + summ
# #         if summ>maxx:
# #             maxx=summ

# print(summ)
# summ = 0
for k in range(34):
    summ = 0
    for sentence in test2.index.values.tolist():
        (shop_id, item_id) = (sentence[0], sentence[1])
        iid = item_dm[item_id]
        sid = shop_dm[shop_id]
        summ = summ + table[iid][sid][k][1]
    print(((k)%12) + 1, summ)

1 13894.0
2 14562.0
3 17846.0
4 14988.0
5 15285.0
6 16531.0
7 17669.0
8 19671.0
9 18520.0
10 19843.0
11 22259.0
12 32155.0
1 26523.0
2 25855.0
3 28559.0
4 25255.0
5 27037.0
6 29634.0
7 30912.0
8 35568.0
9 30658.0
10 33945.0
11 38947.0
12 60033.0
1 45672.0
2 41246.0
3 41986.0
4 35651.0
5 37283.0
6 39126.0
7 42008.0
8 47794.0
9 42015.0
10 44724.0


In [140]:
test = pd.read_csv('all/test.csv')
test = test.set_index(['shop_id', 'item_id'])
test['item_cnt_month'] = 0

test2 = test
test2
for index, sentence in enumerate(test2.index.values.tolist()):
#     print(index, sentence)
    (shop_id, item_id) = (sentence[0], sentence[1])
    val=0
    p = 0
    for j in range(5):
        if predict_test[item_dm[item_id]][shop_dm[shop_id]*5+j] > p:
            p = predict_test[item_dm[item_id]][shop_dm[shop_id]*5+j]
            val = j
#     test.loc[(shop_id, item_id)]['item_cnt_month'] = predict_test[item_dm[item_id]][shop_dm[shop_id]]
    test.loc[(shop_id, item_id)]['item_cnt_month'] = val
    
    

test = test.reset_index().drop(['shop_id', 'item_id'], axis=1)
test.to_csv('submission2.csv', index=False)


In [68]:
import math
from sklearn.metrics import mean_squared_error

# make predictions
# predict_train = model.predict(x_train)
predict_val = model.predict(x_val)
# invert predictions
# predict_train = cnt_scaler.inverse_transform(predict_train)
# yy_train = cnt_scaler.inverse_transform(y_train)
predict_val = cnt_scaler.inverse_transform(predict_val)
yy_val = cnt_scaler.inverse_transform(y_val)
# calculate root mean squared error
# trainScore = math.sqrt(mean_squared_error(predict_train, yy_train))
# print('Train Score: %.2f RMSE' % (trainScore))
valScore = math.sqrt(mean_squared_error(predict_val, yy_val))
print('Test Score: %.2f RMSE' % (valScore))
#For 1 epoch
# Train Score: 2.31 RMSE
# Test Score: 1.42 RMSE
# 6 epoch
# Train Score: 1.89 RMSE
# Test Score: 1.98 RMSE
# 10 epoch
# Train Score: 1.53 RMSE
# Test Score: 2.23 RMSE
# 10 epoch
# 16 LSTM
# Train Score: 1.93 RMSE
# Test Score: 1.90 RMSE
# 10 epoch
# 8 LSTM
# Train Score: 2.31 RMSE
# Test Score: 1.77 RMSE

# 4 LSTM
# Test Score: 1.52 RMSE

# 1 LSTM
# Test Score: 0.98 RMSE

# 4 Dense, size 16, 10 epoch
# 0.82


# 5 Dense, size 16, 12 epoch
# Test Score: 1.36 RMSE

# Dense 16, 8, 8, 8, 8, 
# epoch 8
# Test Score: 0.58 RMSE


Test Score: 1.43 RMSE


In [38]:
print(len(x_fin))
print(len(y_fin))
# print(len(y_train))
# print(len(y_val))

134126
134126


In [39]:
print('Build model...')
model = Sequential()
# model.add(LSTM(1, input_shape=(maxlen, length)))
# model.add(Dense(1, activation='relu'))


# model.add(Dense(64, input_dim=length))
model.add(Dense(16, input_dim=(length)))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(Dropout(0.5))

model.add(Dense(8, input_dim=(length)))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(Dropout(0.5))

model.add(Dense(8, input_dim=(length)))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(Dropout(0.5))

model.add(Dense(8, input_dim=(length)))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(Dropout(0.5))

model.add(Dense(8, input_dim=(length)))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(Dropout(0.5))
# model.add(Dense(16))
# model.add(BatchNormalization())
# model.add(LeakyReLU())
model.add(Dense(1))
# model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.005)
model.compile(loss='mean_squared_error', optimizer=optimizer)

model.fit(x_fin, y_fin, batch_size=2048, epochs=5)

Build model...
Epoch 1/5
134126/134126 [==============================] - 5s 39us/step - loss: 0.2463
Epoch 2/5
134126/134126 [==============================] - 3s 25us/step - loss: 0.1808
Epoch 3/5
134126/134126 [==============================] - 3s 25us/step - loss: 0.1682
Epoch 4/5
134126/134126 [==============================] - 3s 25us/step - loss: 0.1671
Epoch 5/5
134126/134126 [==============================] - 3s 25us/step - loss: 0.1524


In [38]:
model.save("my_model.h5") 

In [ ]:
model = keras.models.load_model("my_model.h5") 

In [95]:
del predict_train
del predict_val

In [58]:
model.fit(x_val, y_val, batch_size=2048, epochs=13)

Epoch 1/13
111404/111404 [==============================] - 19s 172us/step - loss: 9.2909
Epoch 2/13
111404/111404 [==============================] - 19s 172us/step - loss: 7.6454
Epoch 3/13
111404/111404 [==============================] - 19s 173us/step - loss: 6.8270
Epoch 4/13
111404/111404 [==============================] - 19s 173us/step - loss: 6.0881
Epoch 5/13
111404/111404 [==============================] - 19s 174us/step - loss: 5.6661
Epoch 6/13
111404/111404 [==============================] - 19s 173us/step - loss: 5.3044
Epoch 7/13
111404/111404 [==============================] - 19s 173us/step - loss: 5.0030
Epoch 8/13
111404/111404 [==============================] - 19s 172us/step - loss: 4.6944
Epoch 9/13
111404/111404 [==============================] - 19s 173us/step - loss: 4.3458
Epoch 10/13
111404/111404 [==============================] - 19s 171us/step - loss: 4.1903
Epoch 11/13
111404/111404 [==============================] - 19s 172us/step - loss: 3.9588
Epoch 12

In [40]:
predict_test = model.predict(x_test)
predict_test = cnt_scaler.inverse_transform(predict_test)

In [41]:

test = pd.read_csv('all/test.csv')
test = test.set_index(['shop_id', 'item_id'])
test['item_cnt_month'] = 0

for index, sentence in enumerate(x_test_o):
    (shop_id, item_id) = (sentence[0]['shop_id'], sentence[0]['item_id'])
    test.loc[(shop_id, item_id)]['item_cnt_month'] = predict_test[index]
    
    

test = test.reset_index().drop(['shop_id', 'item_id'], axis=1)
test.to_csv('submission2.csv', index=False)
